In [1]:
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
    board3d = layers.Input(shape=(14, 8, 8))
    x = board3d
    for _ in range(conv_depth):
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, 'relu')(x)
    x = layers.Dense(1, 'sigmoid')(x)

    return models.Model(inputs=board3d, outputs=x)

2024-02-07 15:24:54.088893: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 15:24:54.135450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 15:24:54.135486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 15:24:54.136989: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 15:24:54.143858: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 15:24:54.144343: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
def build_model_residual(conv_size, conv_depth):
    board3d = layers.Input(shape=(14, 8, 8))
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu')(board3d)
    for _ in range(conv_depth):
        previous = x
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Add()([x, previous])
        x = layers.Activation('relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1, 'sigmoid')(x)

    return models.Model(inputs=board3d, outputs=x) 

In [ ]:
from IPython.display import SVG, display
import chess
import chess.svg
from chess_env import ChessEnv
import numpy as np
from tensorflow import keras
from chess_env import ChessEnv

def choose_move(model, state):
        q_values= model.predict(state, axis=0)
        move = np.argmax(q_values)
        return move

game = ChessEnv()
state = game.reset()
model = keras.models.load_model('dqn_chess.h5')
display(SVG(chess.svg.board(board=game.board)))
while not game._is_game_over():
    action = game.next_action(state=state)
    game.step(action)
    next_state, reward, done, info = game.step(action)
    display(SVG(chess.svg.board(board=game.board)))
    print("Chosen move:", list(game.board.legal_moves)[action])
    state = next_state
    print("Legal moves:", game.board.legal_moves)
display(SVG(chess.svg.board(board=game.board)))
print("Game over")

In [12]:
s = game.board.board_fen()
print([game.board])

[Board('2bqkQnr/rp3p1p/2p4P/p3n1p1/6P1/5P2/PP1PP3/RNBQKBNR b KQk - 0 11')]
